In [40]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [41]:
device = torch.device('coda:0' if torch.cuda.is_available() else 'cpu')

In [42]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [43]:
train_dataset = torchvision.datasets.MNIST(root='../../data/', train = True, transform=transforms.ToTensor(),download=True)


In [44]:
test_dataset = torchvision.datasets.MNIST(root='../../data/',train=False,transform = transforms.ToTensor())

In [45]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [46]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [47]:
model = ConvNet(num_classes).to(device)

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)


In [49]:
total_step = len(train_loader)

In [50]:
total_step

600

In [51]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1496
Epoch [1/5], Step [200/600], Loss: 0.1739
Epoch [1/5], Step [300/600], Loss: 0.1026
Epoch [1/5], Step [400/600], Loss: 0.0720
Epoch [1/5], Step [500/600], Loss: 0.0153
Epoch [1/5], Step [600/600], Loss: 0.0403
Epoch [2/5], Step [100/600], Loss: 0.1565
Epoch [2/5], Step [200/600], Loss: 0.0620
Epoch [2/5], Step [300/600], Loss: 0.0474
Epoch [2/5], Step [400/600], Loss: 0.0476
Epoch [2/5], Step [500/600], Loss: 0.0385
Epoch [2/5], Step [600/600], Loss: 0.0572
Epoch [3/5], Step [100/600], Loss: 0.0286
Epoch [3/5], Step [200/600], Loss: 0.0088
Epoch [3/5], Step [300/600], Loss: 0.0135
Epoch [3/5], Step [400/600], Loss: 0.0448
Epoch [3/5], Step [500/600], Loss: 0.0265
Epoch [3/5], Step [600/600], Loss: 0.0625
Epoch [4/5], Step [100/600], Loss: 0.0071
Epoch [4/5], Step [200/600], Loss: 0.0050
Epoch [4/5], Step [300/600], Loss: 0.0095
Epoch [4/5], Step [400/600], Loss: 0.0772
Epoch [4/5], Step [500/600], Loss: 0.0027
Epoch [4/5], Step [600/600], Loss:

In [52]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100* correct/total))

Test Accuracy of the model on the 10000 test images: 99.11 %


In [53]:
torch.save(model.state_dict(),'model_cnn.ckpt')